In [9]:
import pandas as pd
import numpy as np
import openpyxl
import threading

data = pd.read_csv("集市聊天记录.csv")

In [10]:
data = data.iloc[:,[0,2]]
data

,消息时间,消息内容
0,2023/9/1 0:01,收仓鼠粮\nhttps://c.zanao.com/l/1SnsEt\n\n溪溪状元笔记不好...
1,2023/9/1 0:14,想收个二手自行车\nhttps://c.zanao.com/l/1SntE4\n\n英**留...
2,2023/9/1 0:32,卡帕床垫\nhttps://c.zanao.com/l/1Snueg\n\n找羽球搭子\nh...
3,2023/9/1 0:52,22学生公寓限电功率多少\nhttps://c.zanao.com/l/1SnvmP\n\n...
4,2023/9/1 7:12,闲置衣架\nhttps://c.zanao.com/l/1SnwyH\n\n出闲置宿舍五连挂...
...,...,...
4751,2023/10/20 12:00,收生化下册书\nhttps://c.zanao.com/l/1TwT8O\n\n学校买的被子...
4752,2023/10/20 12:16,求一张21号漫展HACG**\nhttps://c.zanao.com/l/1TwUwr\n...
4753,2023/10/20 12:31,东西丢了\nhttps://c.zanao.com/l/1TwWas\n\n谁入党是导员决定...
4754,2023/10/20 12:43,出一个林大毕业纪念盒\nhttps://c.zanao.com/l/1TwWXK\n\n今天...


In [11]:
df = pd.DataFrame(columns=["时间","发帖内容"])


In [12]:
import re

# 定义函数来去除网址
def remove_web(text):
    lines = text.split('\n')
    for i in range(1, len(lines), 3):
        lines[i] = ''
    lines = list(filter(lambda x: x != '', lines))
    return '\n'.join(lines)

def trans(data,df):
    data['消息内容'] = data['消息内容'].apply(remove_web)
    # 遍历原始 DataFrame 的行
    # for index, row in data.iterrows():
    #     timestamp = row['消息时间']
    #     text = row['消息内容']
    #     sub_texts = text.split('\n')  # 使用换行符分割子文本

    # # 将拆分的子文本添加到新 DataFrame
    #     for sub_text in sub_texts:
    #         df = df.append({'时间': timestamp, '发帖内容': sub_text}, ignore_index=True)
    data = data.assign(text=data['消息内容'].str.split('\n')).explode('消息内容')


In [13]:
data['消息内容'] = data['消息内容'].apply(remove_web)
df = data.assign(text=data['消息内容'].str.split('\n'))
df = df.iloc[:,[0,2]]
df

,消息时间,text
0,2023/9/1 0:01,"[收仓鼠粮, 溪溪状元笔记不好意思刚刚留错了联系方式, 高数A2补考, 求问心理问题, 求问求问]"
1,2023/9/1 0:14,"[想收个二手自行车, 英**留学费用]"
2,2023/9/1 0:32,"[卡帕床垫, 找羽球搭子, 6号楼怎么晒被子, 请问八公寓上铺的遮光帘尺寸, 转专业求助, ..."
3,2023/9/1 0:52,"[22学生公寓限电功率多少, 家人们跟我聊聊天]"
4,2023/9/1 7:12,"[闲置衣架, 出闲置宿舍五连挂钩收纳袋置物架, 1号七点多哈西拼车回校, 专业排名掉了, 哈..."
...,...,...
4751,2023/10/20 12:00,"[收生化下册书, 学校买的被子大概是多重多宽的啊, 找充电宝, 抱歉抱歉, 求书23版马原毛..."
4752,2023/10/20 12:16,"[求一张21号漫展HACG**, 收一张21号的漫展**, 有偿求讲大一高数, 染发家属区染..."
4753,2023/10/20 12:31,"[东西丢了, 谁入党是导员决定吗, 求问现在还有哪些校组织招人吗, 有偿借Applewatc..."
4754,2023/10/20 12:43,"[出一个林大毕业纪念盒, 今天路过高中, 收明天21号漫展**, 出HACG**一张, 出鸡..."


In [14]:
df = df.explode("text")
df

,消息时间,text
0,2023/9/1 0:01,收仓鼠粮
0,2023/9/1 0:01,溪溪状元笔记不好意思刚刚留错了联系方式
0,2023/9/1 0:01,高数A2补考
0,2023/9/1 0:01,求问心理问题
0,2023/9/1 0:01,求问求问
...,...,...
4755,2023/10/20 12:54,救大命我就说
4755,2023/10/20 12:54,为什么还没供暖
4755,2023/10/20 12:54,集市捞捞
4755,2023/10/20 12:54,求推荐抗糖抗氧的护肤品


In [15]:
df.rename(columns={'消息时间':'发帖时间','text': '发帖内容'}, inplace=True)
df

,发帖时间,发帖内容
0,2023/9/1 0:01,收仓鼠粮
0,2023/9/1 0:01,溪溪状元笔记不好意思刚刚留错了联系方式
0,2023/9/1 0:01,高数A2补考
0,2023/9/1 0:01,求问心理问题
0,2023/9/1 0:01,求问求问
...,...,...
4755,2023/10/20 12:54,救大命我就说
4755,2023/10/20 12:54,为什么还没供暖
4755,2023/10/20 12:54,集市捞捞
4755,2023/10/20 12:54,求推荐抗糖抗氧的护肤品


In [16]:
df.to_excel("集市帖提取.xlsx",index=False)

In [17]:
def remove_enter(text):
    lines = text.split('\n')
    for i in range(1, len(lines), 3):
        lines[i] = ''
    lines = list(filter(lambda x: x != '', lines))
    return ''.join(lines)

In [18]:
data['消息内容'] = data['消息内容'].apply(remove_enter)
df = data
df.to_excel("集市帖聚合提取.xlsx",index=False)